# Code from BSDE.py 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
import pandas as pd
import math
from scipy.stats import norm
from sklearn.ensemble import RandomForestRegressor
import scipy.stats as stats
from sklearn.neighbors import kneighbors_graph
import copy
import time
import scipy.sparse as sparse
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (7,7) # Make the figures a bit bigger

class BSDE(object):

    def __init__(self,S0, K, T, mu, sigma, q):
        '''

        Parameters
        ==========
        S0             : float
                          positive, initial Stock Value
        mu              : float
                          drift
        K               : float
                          Strike price
        T               : float
                          Maturity time
        sigma           : float
                          volatility
         Returns
        =======
        BSDE : class
        '''
        self.S0 = S0
        self.K = K
        self.T = T
        self.mu = mu
        self.sigma = sigma
        self.q = q

    def generate_paths(self,r,N,m, mode='delta_B'):
        if mode == 'delta_B':
            dt = self.T / m
            S = np.zeros((m + 1, N))
            dB = np.zeros((m + 1, N))
            S[0] = self.S0
            for t in range(1,m + 1):
                X = np.random.standard_normal(size=N)
                S[t] = S[t - 1] * np.exp((self.mu - self.sigma * self.sigma/2) * dt
                                         + self.sigma * np.sqrt(dt) * X)
                dB[t] =  np.sqrt(dt) * X
            return (S, dB)
        elif mode=='B':
            dt = self.T / m
            S = np.zeros((m + 1, N))
            B = np.zeros((m + 1, N))
            S[0] = self.S0
            for t in range(1, m + 1):
                X = np.random.standard_normal(size=N)
                S[t] = S[t - 1] * np.exp((self.mu - self.sigma * self.sigma / 2) * dt
                                         + self.sigma * np.sqrt(dt) * X)
                B[t] = np.sqrt(t) * X
            return (S, B)

    def get_price_mesh(self, N, m, r, R,
                       mode='NN', n_neighbors=10,
                       K1=95., K2=105., oPayoff="call",
                       oType="European", n_picard=0,
                       use_variance_reduction=False,
                       display_plot=False):
        """
        Approach
        ========
        Call T(i,j,t) the transition from S_i at time "t" to S_j at time "t+dt". Then:
        1) P(S_[t+dt] = S_j) \approx (1/N) * sum_i T(i,j,t)  \def P^[t+dt]_j
        2) E[X^[t+dt]_j|S^t_i] \approx (1/N) * sum_j T(i,j,t) * X^[t+dt]_j / P^[t+dt]_j  (basic importance sampling)
        3) Consequently, define
            W(i,j,t) =  T(i,j,t) / P^[t+dt]_j
        and we have
            E[ (something) | filtration_t] = (1/N) * dot_product(W, something)
        4) remark: one can slightly reduce the variance by further normalizing the rows

        """

        def do_BSDE_mesh_update(Y, Y_inc, Z, dB, W, cv, cv_expectation, mode = 'all'):
            """
            W       : weight matrix in usual mesh method
            S_t     : Stock price at time "t"
            S_t_dt  : Stock price at time "t + dt"
            dB      : Brownian increments
            """
            # Regression for Z_t:
            #
            # Z_t = E( Y_[t+dt] * dW_t | filtration_t)  /dt
            #
            if mode == 'all':
                if use_variance_reduction == True:
                    # print("I am here")
                    # use the fact that E[ S_t_dt * dW | F_t] = sigma * S * dt + (higher order term)
                    # cv1 = W * S_t_dt * dB  # control variate
                    cv_std = np.std(cv, axis=1)
                    cv_mean = np.mean(cv, axis=1)
                    cv_centred[:] = (cv - cv_mean[:, np.newaxis]) / cv_std[:, np.newaxis]

                    YdB[:] = W * ((Y - Y_inc) * dB[t])
                    YdB_std[:] = np.std(YdB, axis=1)
                    YdB_mean[:] = np.mean(YdB, axis=1)
                    YdB_centred[:] = (YdB - YdB_mean[:, np.newaxis]) / YdB_std[:, np.newaxis]

                    corr = np.mean(cv_centred * YdB_centred, axis=1)
                    Z[:] = np.mean(YdB - (cv - cv_expectation[:, np.newaxis]) * corr[:, np.newaxis], axis=1) * (1. / dt)
                else:
                    Z[:] = (1. / N) * np.dot(W, (Y - Y_inc) * dB[t]) * (1. / dt)
                driver_measurable = -theta * Z
                driver_stoch = - r * Y - (R - r) * np.minimum(Y - (1. / self.sigma) * Z, 0)

                # Regression for Y_t
                #
                # Y_t = E(Y_[t+dt] + dt * driver | filtration_t)
                #
                Y_inc[:] = (1. / N) * np.dot(W, Y + dt * driver_stoch) + dt * driver_measurable
                # Z[:] = numerical_diff(S[t,:], Y_inc) * S[t,:] * self.sigma
                # print("Inside Z={}".format(Z))
                # print("Inside Y_inc={}".format(Y_inc))
            else:
                if use_variance_reduction == True:
                    # print("I am here")
                    # use the fact that E[ S_t_dt * dW | F_t] = sigma * S * dt + (higher order term)
                    # cv1 = W * S_t_dt * dB  # control variate
                    cv_std = np.std(cv, axis=1)
                    cv_mean = np.mean(cv, axis=1)
                    cv_centred[:] = (cv - cv_mean[:, np.newaxis]) / cv_std[:, np.newaxis]

                    YdB[:] = W * ((Y - Y_inc) * dB[t])
                    YdB_std[:] = np.std(YdB, axis=1)
                    YdB_mean[:] = np.mean(YdB, axis=1)
                    YdB_centred[:] = (YdB - YdB_mean[:, np.newaxis]) / YdB_std[:, np.newaxis]

                    corr = np.mean(cv_centred * YdB_centred, axis=1)
                    Z[:] = np.mean(YdB - (cv - cv_expectation[:, np.newaxis]) * corr[:, np.newaxis], axis=1) * (1. / dt)
                else:
                    Z[:] = (1. / N) * np.dot(W, (Y - Y_inc) * dB[t]) * (1. / dt)

        # Time-step
        dt = self.T / m
        # Discount factor
        df = 1 / (1 + r * dt)
        theta = -(r - self.mu) / self.sigma
        drift_dt = (self.mu - 0.5 * self.sigma ** 2) * dt
        sigma_sqt = self.sigma * np.sqrt(dt)
        gauss_normalization = np.sqrt(2 * np.pi) * sigma_sqt
        # S, dB
        # S, B = self.generate_paths(r, N, m, mode='B')
        S, dB = self.generate_paths(r, N, m)

        # price of the option at time T = Initialization for a call
        if oPayoff == "call":
            Y = np.maximum(S[-1] - self.K, 0)
        elif oPayoff == "put":
            Y = np.maximum(self.K - S[-1], 0)
        elif oPayoff == "call combination":
            Y = np.maximum(S[-1] - K1, 0) - 2 * np.maximum(S[-1] - K2, 0)
        elif oPayoff == "put combination":
            Y = np.maximum(K1 - S[-1], 0) - 2 * np.maximum(K2 - S[-1], 0)

        if mode == 'all':
            if (oType == 'European'):
                # log_S follows a drifted Brownian motion with
                # drift = mu - sigma**2/2
                # volatility = sigma
                # weight matrix
                W = np.zeros([N, N])
                transition_matrix = np.zeros([N, N])
                marginal_vector = np.zeros(N)
                dist_matrix = np.zeros([N, N])
                log_S_start, log_S_end = np.zeros(N), np.zeros(N)
                Z, Y_inc = np.zeros(N), np.zeros(N)
                cv, cv_centred = np.zeros([N, N]), np.zeros([N, N])
                cv_std, cv_mean, cv_expectation = np.zeros(N), np.zeros(N), np.zeros(N)
                YdB, YdB_centred = np.zeros([N, N]), np.zeros([N, N])
                YdB_std, YdB_mean = np.zeros(N), np.zeros(N)
                # Iteration over time backwardly
                for t in range(m - 1, 0, -1):
                    # work on logscale
                    log_S_end = np.log(S[t + 1, :])
                    log_S_start = np.log(S[t, :])
                    # distances matrix
                    dist_matrix = -np.subtract.outer(log_S_start, log_S_end)
                    # transition densities matrix
                    transition_matrix = np.exp(
                        -0.5 * np.square(dist_matrix - drift_dt) / sigma_sqt ** 2) / gauss_normalization
                    # marginals
                    marginal_vector = np.mean(transition_matrix, axis=0)

                    if display_plot:
                        plt.subplot(3, m, t + 1)
                        plt.plot(log_S_end, marginal_vector, "r.")
                        plt.title("Marginal Density at time {}".format(str(t * dt)))

                    # weight matrix
                    W = transition_matrix / marginal_vector
                    # normalize the rows
                    # W_row_sums = W.sum(axis=1)
                    # W = W / W_row_sums[:, np.newaxis]

                    # initial guess for Y_inc is set to zero
                    Y_inc[:] = 0.
                    # use control variate
                    cv[:, :] = W * dB[t] * dB[t]
                    cv_expectation[:] = dt
                    # do BSDE update
                    do_BSDE_mesh_update(Y, Y_inc, Z, dB[t], W,
                                        cv, cv_expectation)
                    # print("After Z={}".format(Z))
                    # print("After Y_inc={}".format(Y_inc))


                    for __ in range(n_picard):
                        cv[:, :] = W * Z * dB[t]
                        cv_expectation[:] = 0
                        do_BSDE_mesh_update(Y, Y_inc, Z, dB[t], W,
                                            cv, cv_expectation)
                    Y[:] = Y_inc

                    if display_plot:
                        plt.subplot(4, m, m + t + 1)
                        plt.plot(S[t, :], Y, "b.")
                        plt.title("Price at time".format(str(t * dt)))

                        plt.subplot(4, m, 2 * m + t + 1)
                        plt.plot(S[t, :], Z, "b.")
                        plt.title("Z at time".format(str(t * dt)))


                        # plt.plot(S[t+1,:], Z_, "r.")

                        # plt.plot(X, Z, 'r.')
                        # plt.show()
            Y_opt = df * np.mean(Y)
            return (Y_opt)

        elif mode == 'NN':
            if (oType == 'European'):
                # log_S follows a drifted Brownian motion with
                # drift = mu - sigma**2/2
                # volatility = sigma
                # weight matrix
                Z, Y_inc = np.zeros(N), np.zeros(N)
                cv, cv_centred = np.zeros([N, N]), np.zeros([N, N])
                cv_std, cv_mean, cv_expectation = np.zeros(N), np.zeros(N), np.zeros(N)
                YdB, YdB_centred = np.zeros([N, N]), np.zeros([N, N])
                YdB_std, YdB_mean = np.zeros(N), np.zeros(N)
                # Iteration over time backwardly
                for t in range(m - 1, 0, -1):
                    # work on logscale
                    log_S_end = np.log(S[t + 1, :])
                    log_S_start = np.log(S[t, :])
                    # distances matrix
                    dist_matrix = - np.subtract.outer(log_S_start, log_S_end)
                    # transition densities matrix
                    transition_matrix = np.exp(
                        -0.5 * np.square(dist_matrix - drift_dt) / sigma_sqt ** 2) / gauss_normalization
                    # marginals
                    marginal_vector = np.mean(transition_matrix, axis=0)

                    if display_plot:
                        plt.subplot(3, m, t + 1)
                        plt.plot(log_S_end, marginal_vector, "r.")
                        plt.title("Marginal Density at time {}".format(str(t * dt)))

                    # weight matrix
                    W = transition_matrix / marginal_vector
                    # normalize the rows
                    NN = kneighbors_graph(log_S_end.reshape(N, 1), n_neighbors,
                                          mode='distance').nonzero()
                    x = NN[0]
                    y = NN[1]
                    W_bar = copy.deepcopy(W)
                    
                    #We give 0 to nearest neighbors in copy of W
                    W_bar[x, y] = 0
                    np.fill_diagonal(W_bar, 0.)
                    
                    #Then the new W is the old one with far neighbors = 0
                    W = W - W_bar
                    
                    #We make use of sparse matrix to optimize memory 
                    W_sparse = sparse.csr_matrix(W)

                    # Regression for Z_t:
                    #
                    # Z_t = E( Y_[t+dt] * dW_t | filtration_t)  /dt
                    #

                    
                    if R != r:
                        Z = (1. / N) * W_sparse.dot(Y * dB[t]) * (1. / dt)
                    else:
                        Z = 0.

                    driver_measurable = - theta * Z
                    driver_stoch = - r * Y - (R - r) * np.minimum(Y - (1. / self.sigma) * Z, 0)
                    Y_inc = (1. / N) * W_sparse.dot(Y + dt * driver_stoch) + dt * driver_measurable
                    # follwing is Majdi's version
                    # delta_B = -np.subtract.outer(B[t,:],B[t+1,:])
                    # Z = (1. / n) * np.sum(W * (delta_B * Y[:,np.newaxis]), axis=1) * (1./ dt)

                    # Regression for Y_t
                    #
                    # Y_t = E(Y_[t+dt] + dt * driver | filtration_t)
                    #

                    # driver = -theta * Z - r * Y - (R-r) * np.minimum(Y - (1. / self.sigma) * Z, 0)
                    #   decompose driver as
                    #   driver = driver_measurable + driver_stoch
                    # where "driver_measurable" is already F_t measurable


                    # Y_inc = np.dot(W, Y + dt * driver)
                    for __ in range(n_picard):
                        # update Z
                        # 1sr version
                        if R != r:
                            Z = (1. / N) * W_sparse.dot((Y - Y_inc) * dB[t]) * (1. / dt)
                        else:
                            Z = 0.
                        # Majdi's version
                        # delta_B = -np.subtract.outer(B[t,:],B[t+1,:])
                        # Z = np.sum(W * delta_B, axis=1) * (1./ dt)
                        # update Z
                        driver_measurable = -theta * Z
                        driver_stoch = - r * Y - (R - r) * np.minimum(Y - (1. / self.sigma) * Z, 0)
                        Y_inc = (1. / N) * W_sparse.dot(Y + dt * driver_stoch) + dt * driver_measurable
                    Y = Y_inc

                    if display_plot:
                        plt.subplot(3, m, m + t + 1)
                        plt.plot(S[t, :], Y, "b.")
                        plt.title("Price at time".format(str(t * dt)))

                        plt.subplot(3, m, 2 * m + t + 1)
                        plt.plot(S[t, :], Z, "b.")
                        plt.title("Z at time".format(str(t * dt)))

                        # plt.plot(X, Z, 'r.')
                        # plt.show()
            Y_opt = df * np.mean(Y)
            return Y_opt

### Example 1 : 1 dim Bid-Ask (expected price : 7.15)

In [4]:
import time

In [7]:
T = 0.5
m_discretization = 10
K = 100
S0 = 100
sigma = 0.2
r = 0.04
N_particles = 1000
n_neighbors = 999
mu = 0.06
R = 0.06
q = 0.

start = time.time()

test = BSDE (S0, K, T, mu, sigma, q)
price_mesh = test.get_price_mesh(N_particles, m_discretization, r, R, n_picard = 0,
                                 mode = 'NN', display_plot = False, 
                                 n_neighbors=n_neighbors)
elapsed= time.time() - start
print("Price with method of Mesh Nearest Neighbors is {}, it took {}s, computing {} particles and\
 taking {} neighbors".format(price_mesh, elapsed, N_particles, n_neighbors))

Price with method of Mesh Nearest Neighbors is 7.1844083098664635, it took 8.936954021453857s, computing 1000 particles andtaking 999 neighbors


### Example 2 :  20 dim geometric call option

In [22]:
T = 0.25
m = 4
p = 20
K = 95.
r = 0.06
R = 0.06
M = np.eye(p)
S_init = 100.
mu = 0.05
sigma = 0.2
N = 1000
Q = 0.

test_hd = BsdeHD(T, K, M, mu, Q, sigma, S_init, r, R)
price_mesh_hd_nn = test_hd.get_price_mesh(N, m, option_type = 'call', option_payoff = 'geometric',
                                          n_picard=0, n_neighbors= 100, mode='NN')

print("Mesh Fast = {}".format(price_mesh_hd_nn))
print("BS-comparison = {}".format(test_hd.get_comparison_bs(95., p)))

Mesh Fast = 3.5387908529399144
BS-comparison = 5.942417772333471
